In [56]:
import pandas as pd
import numpy as np
from google.colab import drive
import warnings
warnings.filterwarnings('ignore')
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [57]:
df = pd.read_csv('/content/drive/MyDrive/Notebooks/prices.csv')
df.head(5)

,date,symbol,open,close,low,high,volume
0,1/5/2016 0:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600
1,1/6/2016 0:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400
2,1/7/2016 0:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500
3,1/8/2016 0:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300
4,1/11/2016 0:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600


In [58]:
df.isna().sum()

,0
date,0
symbol,0
open,0
close,0
low,0
high,0
volume,0


In [59]:
df['symbol'].unique()

array(['WLTW', 'A', 'AAL', 'AAP', 'AAPL', 'ABC', 'ABT', 'ACN', 'ADBE',
       'ADI', 'ADM', 'ADP', 'ADS', 'ADSK', 'AEE', 'AEP', 'AES', 'AET',
       'AFL', 'AGN', 'AIG', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALK', 'ALL',
       'ALXN', 'AMAT', 'AME', 'AMG', 'AMGN', 'AMP', 'AMT', 'AMZN', 'AN',
       'ANTM', 'AON', 'APA', 'APC', 'APD', 'APH', 'ARNC', 'ATVI', 'AVB',
       'AVGO', 'AVY', 'AWK', 'AXP', 'AYI', 'AZO', 'BA', 'BAC', 'BAX',
       'BBBY', 'BBT', 'BBY', 'BCR', 'BDX', 'BEN', 'BHI', 'BIIB', 'BK',
       'BLK', 'BLL', 'BMY', 'BSX', 'BWA', 'BXP', 'C', 'CA', 'CAG', 'CAH',
       'CAT', 'CB', 'CBG', 'CBS', 'CCI', 'CCL', 'CELG', 'CERN', 'CF',
       'CHD', 'CHK', 'CHRW', 'CI', 'CINF', 'CL', 'CLX', 'CMA', 'CMCSA',
       'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF', 'COG', 'COH',
       'COL', 'COO', 'COP', 'COST', 'CPB', 'CRM', 'CSCO', 'CSX', 'CTAS',
       'CTL', 'CTSH', 'CTXS', 'CVS', 'CVX', 'CXO', 'D', 'DAL', 'DD', 'DE',
       'DFS', 'DG', 'DGX', 'DHI', 'DHR', 'DIS', 'DISCA', 'DISCK', 

In [60]:
df = df[df['symbol'] == 'AMZN']
x = df['close']
x.head()

,close
284,133.899994
751,134.690002
1219,132.250000
1687,130.000000
2155,133.520004


In [61]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
x = scaler.fit_transform(np.array(x).reshape(-1,1))

In [62]:
x

array([[0.03437308],
       [0.03544683],
       [0.03213048],
       ...,
       [0.90182809],
       [0.8923412 ],
       [0.87157324]])

In [63]:
X_train = x[:1200]
X_test = x[1200:]

In [64]:
X_train.shape,X_test.shape

((1200, 1), (562, 1))

In [65]:
X_train_LSTM = []
y_train_LSTM = []
for i in range(100, X_train.shape[0]):
  X_train_LSTM.append(X_train[i-100:i])
  y_train_LSTM.append(X_train[i])

In [66]:
X_test_LSTM = []
y_test_LSTM = []
for i in range(100, X_test.shape[0]):
  X_test_LSTM.append(X_train[i-100:i])
  y_test_LSTM.append(X_train[i])

In [68]:
X_train_LSTM = np.array(X_train_LSTM)
y_train_LSTM = np.array(y_train_LSTM)
X_test_LSTM = np.array(X_test_LSTM)
y_test_LSTM = np.array(y_test_LSTM)

In [69]:
X_train_LSTM = np.reshape(X_train_LSTM, (X_train_LSTM.shape[0], X_train_LSTM.shape[1], 1))
X_test_LSTM = np.reshape(X_test_LSTM, (X_test_LSTM.shape[0], X_test_LSTM.shape[1], 1))

In [70]:
X_train_LSTM.shape,y_train_LSTM.shape

((1100, 100, 1), (1100, 1))

In [72]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

model = Sequential([
    LSTM(units=128, return_sequences=True, input_shape=(X_train_LSTM.shape[1], 1)),
    Dropout(0.2),

    LSTM(units=64, return_sequences=True, input_shape=(X_train_LSTM.shape[1], 1)),
    Dropout(0.2),

    LSTM(units=32, return_sequences=True, input_shape=(X_train_LSTM.shape[1], 1)),
    Dropout(0.2),

    Dense(16, activation='relu'),
    Dropout(0.2),

    Dense(1)
])

In [73]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train_LSTM, y_train_LSTM, epochs=10, batch_size=32,validation_split=0.2)

Epoch 1/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 13s 297ms/step - loss: 0.0128 - val_loss: 0.0106
Epoch 2/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 9s 300ms/step - loss: 0.0033 - val_loss: 0.0068
Epoch 3/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 11s 325ms/step - loss: 0.0025 - val_loss: 0.0063
Epoch 4/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 8s 252ms/step - loss: 0.0023 - val_loss: 0.0060
Epoch 5/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 9s 312ms/step - loss: 0.0021 - val_loss: 0.0061
Epoch 6/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 8s 217ms/step - loss: 0.0021 - val_loss: 0.0061
Epoch 7/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 8s 300ms/step - loss: 0.0020 - val_loss: 0.0064
Epoch 8/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 6s 216ms/step - loss: 0.0017 - val_loss: 0.0066
Epoch 9/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 10s 214ms/step - loss: 0.0017 - val_loss: 0.0069
Epoch 10/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 12s 275ms/step - loss: 0.0017 - val_loss: 0.0062


In [74]:
model.evaluate(X_test_LSTM, y_test_LSTM)

15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - loss: 0.0015


0.0012462349841371179